In [2]:
%%writefile brutegui.py

import os
import time
import numpy as np
from random import randrange
from Crypto.Cipher import AES
import base64
from IPython.html import widgets
from IPython.display import display
import datetime
import hwaes
from binascii import unhexlify
from binascii import hexlify
from multiprocessing import Process, Queue

import partialkey
import aescore
import math
import cluster

np.set_printoptions(formatter={'int':hex})


def encrypt(key, data):    
    mode = AES.MODE_ECB
    aes = AES.new(key, mode)
    aes.use_aesni=True
    #crypto = aes.encrypt(data)
    #print np.frombuffer(crypto, dtype=np.uint64)
    #data = np.frombuffer(data, dtype=np.uint64)
    crypto = aes.encrypt(data)
    #print np.frombuffer(crypto, dtype=np.uint64)
    #print np.getbuffer(data)
    #print data
    return base64.b64encode(crypto)

def search_key_test_soft1(key, data, crypto, searchspace = 0, maskbits = 0):
    keysize = 256
    datasize = 128
    base = 64
    mode = AES.MODE_ECB
    testkey = key.copy()
    counter = 0
    while counter < 2**maskbits:
        testkey[0] = int(key[0]) + (counter << searchspace)
        counter = counter + 1
        aes = AES.new(testkey, mode)
        #aes.use_aesni=True
        testencryptedbuffer = aes.encrypt(data)
        
        if testencryptedbuffer == crypto:
            return testkey
    return 0

def search_key_test_soft(key, data, crypto, searchspace = 0, maskbits = 0):    
    keysize = 256
    datasize = 128
    base = 64
    mode = AES.MODE_ECB
    key = np.getbuffer(key)
    key = key[::-1]
    key = int(hexlify(key), 16)
    counter = 0
    fmt = '%%0%dx' % (33)
    while counter != 2**maskbits:
        key1 = key ^ (counter << searchspace)
        if counter % 2**(maskbits-7) == 0:
            print key1
        testkey = unhexlify(fmt % key1)
        aes = AES.new(testkey[::-1], mode)
        aes.use_aesni=True
        testencryptedbuffer = aes.encrypt(data)
        if testencryptedbuffer == crypto:
            return hex(key1)
        counter = counter + 1
    return 0

    
def BruteGUI():
    keysize = 256
    base = 64
    key = np.array([randrange(2**base) for i in range (keysize/base)], dtype=np.uint64)
    #key[0] = 0xFFFFFFFFFFFFFFFF
    #key[1] = 0xFFFFFFFFFFFFFFFF
    #key[2] = 0xFFFFFFFFFFFFFFFF
    #key[3] = 0xFFFFFFFFFFFFFFFF

    Searchkey = partialkey.searchkey()
    keybox = [widgets.Text(description='Key%d:' % j, value= "%x" % i) for i, j in zip(key, range(len(key)))]
    partialkeybox = [widgets.Text(description='partialkey%d:' % j) for j in range(len(key))]

    text = "Abhishek Bajpai."
    textbox = widgets.Text(description='text:', value= text)
    cryptbox = widgets.Text(description='CryptText:')
    searchbitbox = widgets.Text(description='Search Bit:', value= '0')
    maskbitsbox = widgets.Text(description='Mask Bits:', value= '30')

    resultbox = widgets.Textarea(description='result:')
    jobbox = widgets.Textarea(description='jobs:')

    encryptbutton = widgets.Button(description='encrypt')
    searchbutton = widgets.Button(description='search')
    copybutton = widgets.Button(description='create partial key')
    resetcluster = widgets.Button(description='Reset Cluster')

    hwsel = widgets.Select(description='HW:', options=['x86', 'FPGA'],)
    progress = widgets.FloatProgress(value=0, min=0.0, max=100.0, step=0.1, description='Searching:',)
    progress.visible=False

    container0 = widgets.VBox(children=keybox)
    container2 = widgets.VBox(children=partialkeybox)
    container3 = widgets.HBox(children=[container0, container2])
    container4 = widgets.HBox(children=[searchbitbox, maskbitsbox])

    container1 = widgets.HBox(children=[encryptbutton, copybutton, searchbutton, resetcluster])
    container = widgets.VBox(children=[container3, textbox, container4, hwsel, container1, cryptbox, resultbox, progress, jobbox])

    def partialsearch(q, Searchkey, key, text, crypto, searchptr, searchsize):
        #Searchkey = partialkey.searchkey()    
        Searchkey.reset()
        print 'key', key
        print 'text', text
        print 'crypto', crypto
        print 'searchptr', searchptr
        print 'searchsize', searchsize
        res, reskey = Searchkey.search(key, text, crypto, q, searchptr = searchptr, searchsize = searchsize)
        #for i in range(10):
        q.put({'res':res, 'reskey':reskey})
        
        
    def encryptbutton_clicked(b):
        key = []
        for i in keybox:
            key.append(int(i.value, 16))
        key = np.array(key, dtype=np.uint64)
        text = str.encode(str(textbox.value))
        text = text[:16]
        text = str.encode(str(text))
        cipher = encrypt(key, text)
        cryptbox.value = cipher

    def copybutton_clicked(b):
        for i, j in zip(keybox, partialkeybox):
            j.value = i.value
        searchbit = int(searchbitbox.value)
        maskbits = int(maskbitsbox.value)
        a = (2**maskbits-1) << (searchbit%base)
        b0 = a & 0xFFFFFFFFFFFFFFFF
        b1 = a >> 64
        c = searchbit/base
        d0 = int(partialkeybox[c].value, 16)
        d1 = int(partialkeybox[c+1].value, 16)
        partialkeybox[c].value = "%x" % ((d0 & b0) ^ d0)
        partialkeybox[c+1].value = "%x" % ((d1 & b1) ^ d1)

    def resetcluster_clicked(b):
        Searchkey.reset()

    def searchbutton_clicked(b):
        key = []
        for i in partialkeybox:
            key.append(int(i.value, 16))
        key = np.array(key, dtype=np.uint64)
        text = str.encode(str(textbox.value))
        text = text[:16]
        cipher = base64.b64decode(cryptbox.value)   
        crypto = base64.b64encode(cipher)
        searchbit = int(searchbitbox.value)
        maskbits = int(maskbitsbox.value)
        resultbox.value = ""
        jobbox.value = ""
        t1 = datetime.datetime.now()
        if hwsel.value == 'x86':
            progress.visible=True
            progress.value= 0
            key = search_key_test_soft(key, text, cipher, searchspace = searchbit, maskbits=maskbits)
            reskey = key
        elif hwsel.value == 'FPGA':
            progress.visible=True
            progress.value= 0
            #print key
            #Searchkey.reset()
            #print [key, text, crypto, searchbit, maskbits]
            
            q = Queue()
            res = 0
            reskey = 0
            watchdog = 0
            p = Process(target=partialsearch, args=(q, Searchkey, key, text, crypto, searchbit, maskbits))
            p.start()
            while 1:
                try:
                    r = q.get(block=False)
                    #print '####', r
                    if 'res' in r.keys():
                        res = r['res']
                        reskey = r['reskey']
                        break
                    if 'group' in r.keys():
                        progress.value= 100.0* (1-(1.0*len(r['group']))/(2**maskbits/2**26))
                        jobbox.value = string = str(r['group'])
                        jobsremaining = str(len(r['group'])) + " jobs remaining :"
                        jobbox.description=jobsremaining
                        watchdog = 0

                except: # (Empty):
                    #print 'watchdog:', watchdog
                    watchdog = watchdog + 1
                    if watchdog == 10:
                        break
                    time.sleep(1)
            p.join()
            #res, reskey = Searchkey.search(key, text, crypto, searchptr = searchbit, searchsize = maskbits)
            #print hex(mask3)
            #print hex(mask4)
        t2 = datetime.datetime.now()
        t3 = t2-t1
        #print counter, i
        
        string = str(reskey) + "\nExecution time: %s seconds" % (t3.total_seconds())
        
        resultbox.value = string
        progress.visible=False

    encryptbutton.on_click(encryptbutton_clicked)
    searchbutton.on_click(searchbutton_clicked)
    resetcluster.on_click(resetcluster_clicked)
    copybutton.on_click(copybutton_clicked)
    display(container)
    

Overwriting brutegui.py


In [10]:
%%writefile encryptgui.py

import os
import time
import numpy as np
from random import randrange
from Crypto.Cipher import AES
import base64
from IPython.html import widgets
from IPython.display import display
import datetime
import hwaes
from binascii import unhexlify
from binascii import hexlify
from multiprocessing import Process, Queue

import partialkey
import aescore
import math
import cluster

def FPGAGUI():
    keysize = 256
    base = 64
    key = np.array([randrange(2**base) for i in range (keysize/base)], dtype=np.uint64)
    #key[0] = 0xFFFFFFFFFFFFFFFF
    #key[1] = 0xFFFFFFFFFFFFFFFF
    #key[2] = 0xFFFFFFFFFFFFFFFF
    #key[3] = 0xFFFFFFFFFFFFFFFF
    
    Searchkey = partialkey.searchkey()
    keybox = [widgets.Text(description='Key%d:' % j, value= "%x" % i) for i, j in zip(key, range(len(key)))]
    
    text = "Abhishek Bajpai."
    textbox = widgets.Textarea(description='text:', value= text)

    cryptbox = widgets.Textarea(description='CryptText:')
    resultbox = widgets.Textarea(description='result:')
    
    
    encryptbutton = widgets.Button(description='encrypt')
    fpgabutton = widgets.Button(description='encrypt with cluster')
    resetcluster = widgets.Button(description='Reset Cluster')
    decryptbutton = widgets.Button(description='decrypt')
    
    container0 = widgets.VBox(children=keybox)
    
    container1 = widgets.HBox(children=[encryptbutton, decryptbutton, fpgabutton, resetcluster])
    container = widgets.VBox(children=[container0, textbox, container1, cryptbox, resultbox])
    
    Cluster = cluster.cluster()
    Cluster.conf(4)
        
    def resetcluster_clicked(b):
        Cluster.reset()
        Cluster.conf(4)
        cryptbox.value = ''
        resultbox.value = ''
        
    def encryptbutton_clicked(b):
        key = []
        for i in keybox:
            key.append(int(i.value, 16))
        key = np.array(key, dtype=np.uint64)
        text = str.encode(str(textbox.value))
        if len(text)%16:
            for i in range(16 - (len(text)%16)):
                text = text + ' '
        
        mode = AES.MODE_ECB
        aes = AES.new(key, mode)
        aes.use_aesni=True
        data = np.frombuffer(text, dtype=np.uint64)
        crypto = aes.encrypt(data)
        cryptbox.value = base64.b64encode(crypto)
        
    def decryptbutton_clicked(b):
        key = []
        for i in keybox:
            key.append(int(i.value, 16))
        key = np.array(key, dtype=np.uint64)
        crypto = str.encode(str(cryptbox.value))
        
        mode = AES.MODE_ECB
        aes = AES.new(key, mode)
        aes.use_aesni=True
        resultbox.value = aes.decrypt(base64.b64decode(crypto))
        
    def FPGAbutton_clicked(b):        
        N = 32
        key = []
        for i in keybox:
            key.append(int(i.value, 16))
        key = np.array(key, dtype=np.uint64)
        text = str.encode(str(textbox.value))
        if len(text)%8:
            for i in range(8 - (len(text)%8)):
                text = text + ' '
        
        data = np.frombuffer(text, dtype=np.uint64)
        
        space = ord(' ')
        spacein64 =  ((space << 56) ^  (space << 48) ^  (space << 40) ^  (space << 32) ^ 
                      (space << 24) ^  (space << 16) ^ (space << 8) ^ space)
        if len(data)%N:
            for i in range(N - (len(data)%N)): #if len(data)%2:
                data = np.append([data], np.array([spacein64], dtype=np.uint64))
        
        fpgadata = np.array([], dtype=np.uint64)
        for i in range(len(data)/(N)):
            packetid0, databuff0 = aescore.burst_singlekey(key, data[N*i:N*(i+1)])
            Cluster.sendpackets(databuff0, source=0x0000, sink=0x0001)
            for j in range(10):
                #time.sleep(1/10)
                retbuff = Cluster.getpackets([packetid0])
                if len(retbuff):
                    fpgadata = np.append(fpgadata, retbuff[0]['packet'][1:])
        cryptbox.value = base64.b64encode(fpgadata)
    
    fpgabutton.on_click(FPGAbutton_clicked)
    encryptbutton.on_click(encryptbutton_clicked)
    resetcluster.on_click(resetcluster_clicked)
    decryptbutton.on_click(decryptbutton_clicked)
    display(container)

Overwriting encryptgui.py


In [3]:
print t3.microseconds
print t3.total_seconds()

NameError: name 't3' is not defined

In [ ]:
from Crypto.Cipher import AES

import numpy as np
from random import randrange
keysize = 256
base = 64

mode = AES.MODE_ECB
key = np.array([randrange(2**base) for i in range (keysize/base)], dtype=np.uint64)
aes = AES.new(key, mode)
aes.use_aesni=True
#crypto = aes.encrypt(data)
#print np.frombuffer(crypto, dtype=np.uint64)
#data = np.frombuffer(data, dtype=np.uint64)
#crypto = aes.encrypt(data)